### IMPORT LIBRARY

In [75]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [76]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [77]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [78]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2021
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2018
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2019
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2016
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2020
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2017


In [79]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2021/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2018/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2019/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q3
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q4
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q2
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2016/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2020/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q2
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2017/Q4


In [80]:
quarter = ['Q1']#Chọn năm cần import
year = ['2023']#Chọn quý cần import
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub1:
    if i.split('/')[-2] in year and i.split('/')[-1] in quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q1/FDI_20230331.xlsx


In [81]:
url = []
for i in list_file:
    data_key = i.split('/')[-1].split('.')[0].split('_')[-1]
    if str(data_key) not in ('x'):
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q1/FDI_20230331.xlsx']

In [ ]:
def check_dictionary(df_dict, file_name, data, column_name, parameter, sector, cnxn, sp_object):
    lower_function = lambda x: x.strip().lower() if isinstance(x, str) else x
    if parameter in ['City', 'District', 'Status', 'Indicator'
                     , 'Country', 'Industry'
                    ]:
        raw_parameter = pd.read_sql(f'select * from GENERAL.{parameter}_Dictionary',cnxn)
        raw_parameter[f'Raw_{parameter}'] = raw_parameter[f'Raw_{parameter}'].apply(lower_function)
    elif parameter == 'Grade':
        if sector in ['HOTEL', 'OFFICE', 'APARTMENT'
                      , 'APT', 'SA', 'SERVICED_APARTMENT'
                     ]:
            raw_parameter = pd.read_sql(f"select * from GENERAL.Project_{parameter}_Dictionary",cnxn)
            raw_parameter[f'Raw_{parameter}'] = raw_parameter[f'Raw_{parameter}'].apply(lower_function)
        else:
            return data, df_dict
    elif parameter in ['Type', 'Sub_Type']:
        if sector in ['RETAIL', 'VLTH', 'APARTMENT'
                      , 'APT', 'IP', 'TENANT'
                     ]:
            raw_parameter = pd.read_sql(f"select * from GENERAL.Project_{parameter}_Dictionary WHERE Sector = '{sector}' ",cnxn)
            raw_parameter[f'Raw_{parameter}'] = raw_parameter[f'Raw_{parameter}'].apply(lower_function)
        else:
            return data, df_dict
    elif parameter in ['Project_Name', 'Investor_Nationality', 'Industry_Lv1', 'Investment_Form'
                      , 'Infrastructure_Level_1', 'Infrastructure_Level_2'
                      ]:
        if sector in ['FDI', 'INFRA']:
            header_BIHub = 'share_point_BIHub'
            config_BIHub = read_config_json(config_path, header_BIHub)
            BIHub = SharePoint(config_BIHub)
            raw_parameter = eval(sp_object).get_file(f'/sites/BIHub/Shared Documents/Advisory Data/{sector}/Mapping/{parameter}_Dictionary.xlsx')
            raw_parameter[f'Raw_{parameter}'] = remove_unicode(raw_parameter[f'Raw_{parameter}']).apply(lower_function)
            raw_parameter = raw_parameter.drop_duplicates(subset=[f'Raw_{parameter}'], keep='last')
        else:
            return data, df_dict     
    else:
        print(colored('Unknown parameter in check dictionary section','yellow'))
        return data, df_dict 
    
    data[f'{column_name}']= data[f'{column_name}'].apply(lower_function)
    # data[f'Convert_{parameter}'] = pd.merge(data, raw_parameter
    #                                         , how='left'
    #                                         , left_on=f'{column_name}'
    #                                         , right_on=f'Raw_{parameter}')[f'Cleaned_{parameter}']

    # Hieu update - 31-05-2023
    get_cleaned_type = lambda x: raw_parameter.loc[raw_parameter[f'Raw_{parameter}'] == x,f'Cleaned_{parameter}'].values[0] #fix lỗi duplicate của hàm merge trên\
    print(get_cleaned_type)
    data[f'Convert_{parameter}'] = data[f'{column_name}'].map(get_cleaned_type)
    ## End - Hieu update - 31-05-2023

    parameter_not_in_dict = data[f'{column_name}'][data[f'Convert_{parameter}'].isnull()]
    
    if len(parameter_not_in_dict) != 0:
        temp_df = pd.DataFrame()
        parameter_not_in_dict = list(set(parameter_not_in_dict))
        temp_df.insert(loc=0, column = 'Missing_Values', value = parameter_not_in_dict)
        temp_df.insert(loc=0, column = 'File_Name', value = f'{file_name}')
        temp_df.insert(loc=0, column = 'Flag', value = f'{parameter}')
        df_dict = pd.concat([df_dict, temp_df], axis=0)
        return data, df_dict
    else:
        if parameter == 'Project_Name':
            data[f'{column_name}'] = data[f'{column_name}'].str.title()
            data[f'Convert_{parameter}'] = data[f'Convert_{parameter}'].str.title()
            data.rename(columns = {f'Convert_{parameter}':f'{parameter}_Eng'}, inplace = True)
        else:
            data[f'{column_name}'] = data[f'Convert_{parameter}']
            data = data.drop(columns = [f'Convert_{parameter}'])
        return data, df_dict
    

### IMPORT DATA

In [83]:
'''Prepare ingredients'''
columns_that_need_unidecode=['Project_Name', 'Business_Name', 'Investor_Name', 'Investor_Nationality'
                             , 'Industry_Lv1', 'Investment_Form', 'City', 'District'
                            ]
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat_fdi = pd.DataFrame()
df_flat_fdi = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file
    #-------------------------------------------------------
    '''Validation step'''
    #Remove unfortmated values
    #data = remove_unformated_character(type_sector = 2, sector = sector, file_name = file_name, data = data)
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Project_Name', 'Investor_Nationality', 'Industry_Lv1', 'Investment_Form']
    lst_cls = ['City', 'District', 'Project_Name', 'Investor_Nationality', 'Industry_Lv1', 'Investment_Form']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
        df_temp_flat_fdi = pd.concat([df_temp_flat_fdi, data], axis=0)
        df_flat_fdi = tracking_flat_file(df_temp_flat_fdi, sector)
        insert_to_fresh(file_url, data, engine)
    else:
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

FDI
parameter_not_in_dict:  Series([], Name: City, dtype: object)
FDI
parameter_not_in_dict:  595    None
616    None
637    None
638    None
Name: District, dtype: object
Toang
FDI
parameter_not_in_dict:  0                     nha may cong nghe chinh xac fulian
1         cong ty tnhh cong nghe thong minh goertek vina
2      san xuat san pham khoa chot va dap dinh hinh b...
3      du an dau tu xay dung va kinh doanh ha tang kh...
4                du an mo rong nang luc san xuat pandora
5                          cong ty tnhh autoliv viet nam
6      nha may cong ty tnhh nwestern solar viet nam 1...
7         du an san xuat cung ung vat lieu ung dung hiuv
8                     du an cong nghiep titan bac ninh 2
9                   du an nam thuan logistic park (lo n)
10     du an dau tu cua cong ty tnhh ce link hai duon...
11                         du an khai thac tau container
12     du an thong minh san xuat vanh xe bang hop kim...
13                        cong ty co phan sp ho chi m

100%|██████████| 1/1 [00:20<00:00, 21.00s/it]

parameter_not_in_dict:  Series([], Name: Investment_Form, dtype: object)


### SEND EMAIL

In [84]:
list_df_flat = [df_flat_fdi]
if len(df_dict) != 0:
    print(colored('Missing values in dictionary','yellow'))
    df_noti_html = convert_df_to_html(type_html = 2, df = df_dict, cnxn = engine)
    to_email = ['hcmcbi-intern04@savills.com.vn']
    run_email(email_type = 2, user_email = to_email, df_noti_html = df_noti_html)
else:
    print('Imported data succesfully')
    df_flat_html, df_query_html = convert_df_to_html(type_html = 3, list_df = list_df_flat, type_sector = 2, cnxn = engine)
    to_email = ['hcmcbi-intern04@savills.com.vn']
    run_email(type_sector = 'FDI', email_type = 3, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

Missing values in dictionary


Sent email successfully


### TRACKING AUDIT

In [85]:
insert_to_tracking(list_df_flat, 'Tracking_FDI', engine)

Inserted audit tracking sucessfully
